In [ ]:
# Reset after installing gensim
!pip install vaderSentiment
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import spacy
from datetime import datetime
import re
import string
import unicodedata
import random
from gensim import corpora, models

### Load Reddit Data

In [ ]:
url = 'https://raw.githubusercontent.com/EunCheolChoi0123/COMM557Tutorial/refs/heads/main/Tutorial%203%20Reddit%20Scraper%20and%20Network/datascience-new-comment.csv'
df  = pd.read_csv(url)
df = df[~df.body.isna()]
docs = df.body.to_list()

### Preprocessing text

In [ ]:
import nltk
from nltk import ToktokTokenizer
import spacy
import re, unicodedata


nlp = spacy.load('en_core_web_sm')

### Run this the first time
nltk.download('stopwords')
nltk.download('punkt')

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')


def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

def strip_links(text: str) -> str:
    SIMPLE_LINK_RE = re.compile(r'https?://\S+|www\.\S+')
    return SIMPLE_LINK_RE.sub(' ', text)

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

def noun_only(text, include_proper=True, lowercase=True, unique=False):
    if not isinstance(text, str) or not text.strip():
        return ""
    doc = nlp(text)
    keep_pos = {"NOUN", "PROPN"} if include_proper else {"NOUN"}
    out = [
        (tok.text.lower() if lowercase else tok.text)
        for tok in doc
        if tok.pos_ in keep_pos and tok.is_alpha and not tok.is_stop
    ]
    if unique:
        out = list(dict.fromkeys(out))
    return " ".join(out)


def process_posts(post):
    post = remove_stopwords(post)
    post = strip_links(post)
    post = remove_special_characters(post)
    post = lemmatize_text(post)
    post = noun_only(post)

    return post

df.loc[:, 'body'] = df.body.apply(process_posts)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
processed_docs = [process_posts(doc) for doc in docs]

In [ ]:
processed_docs[0]

'starting point pivot ai'

### Tokenizing and Corpus Creation

In [ ]:
# Create corupus of all words

words_corpus = []

# all comments are splitted as lists
for elem in processed_docs:
    words_corpus.append(elem.lower().split())
print(len(words_corpus))

# gensim corpora dictionary: number of entire unique words
dictionary = corpora.Dictionary(words_corpus)
print(len(dictionary))

109
696


In [ ]:
# number of docs and number of words
dictionary.num_docs, dictionary.num_pos

(109, 1573)

In [ ]:
# get token-id mapping
dictionary.token2id

{'ai': 0,
 'pivot': 1,
 'point': 2,
 'starting': 3,
 'building': 4,
 'c': 5,
 'datum': 6,
 'mca': 7,
 'ml': 8,
 'portfolio': 9,
 'problemsolve': 10,
 'programming': 11,
 'project': 12,
 'python': 13,
 'science': 14,
 'september': 15,
 'skill': 16,
 'sql': 17,
 'student': 18,
 'suggestion': 19,
 'collection': 20,
 'detail': 21,
 'thank': 22,
 'thread': 23,
 'use': 24,
 'analyticsds': 25,
 'break': 26,
 'budget': 27,
 'capital': 28,
 'company': 29,
 'culture': 30,
 'example': 31,
 'experiment': 32,
 'ground': 33,
 'head': 34,
 'idea': 35,
 'leadership': 36,
 'minute': 37,
 'month': 38,
 'napkin': 39,
 'nerd': 40,
 'number': 41,
 'ofc': 42,
 'partner': 43,
 'people': 44,
 'process': 45,
 'product': 46,
 'proposal': 47,
 'pushback': 48,
 'redone': 49,
 'stuff': 50,
 'taker': 51,
 'team': 52,
 'tell': 53,
 'term': 54,
 'thing': 55,
 'ticket': 56,
 'time': 57,
 'timeline': 58,
 'trash': 59,
 'view': 60,
 'week': 61,
 'willing': 62,
 'working': 63,
 'boilerplate': 64,
 'code': 65,
 'codexclau

In [ ]:
dictionary.most_common(10)

[('job', 42),
 ('science', 29),
 ('datum', 26),
 ('year', 22),
 ('thing', 19),
 ('time', 19),
 ('project', 18),
 ('lightning', 18),
 ('lot', 16),
 ('work', 16)]

In [ ]:
# get bag-of-words repesentation of documents: list of (token_id, token_count) tuples
dictionary.doc2bow(words_corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1)]

## Topic Modeling
# https://radimrehurek.com/gensim/models/ldamodel.html

In [ ]:
?dictionary.filter_extremes

In [ ]:
# filtering outliers and putting into bag-of-words

dictionary = corpora.Dictionary(words_corpus)
dictionary.filter_extremes(no_below=10, no_above=0.2, keep_n=1000)

corpus_bow = [dictionary.doc2bow(text) for text in words_corpus]

# Term Frequency - Inverse Document Frequency

from gensim import corpora, models

tfidf = models.TfidfModel(corpus_bow)
corpus_tfidf = tfidf[corpus_bow]

In [ ]:
corpus_tfidf.corpus

[[],
 [(0, 1), (1, 1), (2, 1)],
 [],
 [(3, 1)],
 [],
 [(0, 2), (4, 1), (5, 3), (6, 1)],
 [(3, 1)],
 [],
 [(1, 1), (3, 1), (6, 2), (7, 2)],
 [(1, 1), (3, 1), (5, 1), (8, 1)],
 [],
 [],
 [(6, 1)],
 [],
 [],
 [(9, 1), (10, 1)],
 [(1, 1)],
 [(3, 1)],
 [],
 [(3, 1)],
 [(3, 2), (5, 2), (6, 2), (9, 1)],
 [(6, 2), (9, 1), (10, 1)],
 [(7, 1), (11, 1)],
 [],
 [],
 [(1, 1)],
 [],
 [],
 [],
 [],
 [],
 [(1, 1)],
 [(0, 1)],
 [],
 [(0, 1)],
 [],
 [(0, 1), (5, 1), (9, 1), (11, 2)],
 [(3, 1), (5, 2), (9, 3), (11, 1)],
 [],
 [],
 [],
 [],
 [],
 [(7, 1), (11, 1)],
 [(0, 7), (2, 7), (3, 1), (5, 1), (7, 3), (10, 1)],
 [(0, 3), (2, 3), (6, 3), (7, 4), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(0, 1), (10, 1)],
 [(0, 1), (2, 3), (7, 1), (12, 1)],
 [(0, 1), (2, 2), (8, 1), (10, 1), (12, 1), (13, 1)],
 [(6, 1), (8, 1)],
 [],
 [(8, 1), (12, 1)],
 [(0, 1), (1, 1), (2, 1), (6, 1), (12, 1)],
 [(1, 1), (2, 1), (4, 1), (5, 1), (7, 4), (8, 1), (10, 1), (11, 3), (12, 1)],
 [(0, 1), (2, 6), (7, 2), (8, 1), (13, 2)]

In [ ]:
# Only 5 topics this time for simplicity
num_topics = 5
lda_model_tfidf = models.LdaMulticore(corpus_tfidf, num_topics=num_topics, id2word=dictionary, passes=4, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1, num_words=20):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.365*"use" + 0.288*"thing" + 0.120*"lot" + 0.057*"time" + 0.027*"work" + 0.021*"job" + 0.020*"company" + 0.017*"project" + 0.017*"data" + 0.015*"role" + 0.014*"year" + 0.013*"datum" + 0.013*"experience" + 0.013*"science"
Topic: 1 Word: 0.461*"project" + 0.117*"time" + 0.082*"role" + 0.062*"use" + 0.060*"job" + 0.046*"work" + 0.045*"lot" + 0.022*"experience" + 0.020*"science" + 0.019*"data" + 0.017*"datum" + 0.017*"thing" + 0.017*"year" + 0.017*"company"
Topic: 2 Word: 0.189*"data" + 0.169*"time" + 0.159*"lot" + 0.142*"year" + 0.124*"experience" + 0.057*"project" + 0.049*"company" + 0.028*"thing" + 0.021*"datum" + 0.018*"job" + 0.016*"use" + 0.009*"work" + 0.009*"role" + 0.009*"science"
Topic: 3 Word: 0.247*"job" + 0.223*"datum" + 0.220*"role" + 0.093*"year" + 0.093*"company" + 0.031*"time" + 0.028*"experience" + 0.014*"lot" + 0.012*"work" + 0.012*"project" + 0.008*"thing" + 0.007*"use" + 0.007*"data" + 0.007*"science"
Topic: 4 Word: 0.222*"science" + 0.194*"work" + 0.10

## Visualizing Topic Modeling

In [ ]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 16.0 MB/s eta 0:00:00


In [ ]:
import pyLDAvis
import pyLDAvis.gensim

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# MDS for projecting to two-dimensions
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.088166  0.045978       1        1  24.842782
3     -0.200151  0.050434       2        1  24.158550
2      0.044736 -0.078078       3        1  20.245192
0      0.215994  0.096317       4        1  16.308932
1      0.027587 -0.114651       5        1  14.444545, topic_info=       Term      Freq     Total Category  logprob  loglift
1   project  8.000000  8.000000  Default  14.0000  14.0000
3       use  7.000000  7.000000  Default  13.0000  13.0000
5     thing  6.000000  6.000000  Default  12.0000  12.0000
2   science  6.000000  6.000000  Default  11.0000  11.0000
13     role  7.000000  7.000000  Default  10.0000  10.0000
..      ...       ...       ...      ...      ...      ...
8      data  0.260790  5.634407   Topic5  -3.9665  -1.1381
2   science  0.279462  6.071380   Topic5  -3.8973  -1.1436
10     year  0.228805  6.193079   Topic5  -4.0973  -1.3635
5     thing  0.229425  6.429530   Topic5  -4.0946  -1.3982
0     datum  0.230609  8.464734   Topic5  -4.0895  -1.6681

[84 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
4         1  0.221720     company
4         2  0.443440     company
4         3  0.221720     company
8         1  0.177481        data
8         3  0.709924        data
0         1  0.236274       datum
0         2  0.590686       datum
12        1  0.376481  experience
12        2  0.188241  experience
12        3  0.376481  experience
7         1  0.307711         job
7         2  0.615422         job
7         5  0.102570         job
9         3  0.471640         lot
9         4  0.314426         lot
9         5  0.157213         lot
1         1  0.112787     project
1         3  0.112787     project
1         5  0.676720     project
13        1  0.137324        role
13        2  0.686620        role
13        5  0.137324        role
2         1  0.823536     science
5         1  0.155532       thing
5         3  0.155532       thing
5         4  0.622129       thing
6         1  0.139612        time
6         2  0.139612        time
6         3  0.418837        time
6         4  0.139612        time
6         5  0.279225        time
3         4  0.836401         use
3         5  0.139400         use
11        1  0.817955        work
11        5  0.163591        work
10        1  0.161471        year
10        2  0.322941        year
10        3  0.484412        year, R=14, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 3, 1, 2])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# On Your Own
1. Scrape 1,000 submissions from a subreddit using Tutorial 2 script. You don't have to scrape comments, user data, etc. for this session.
2. Conduct LDA Topic Modeling. How many topics make the best sense in your opinion?
3. (Optional) Name each topics.